<a href="https://colab.research.google.com/github/Stephen-Robbins/Math_of_Finance/blob/main/MATH_176.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimal Bets and Profit for n-Outcome Arbitrage


This code implements the solution to the following theorem:

**Theorem 2 (Optimal Bets and Profit for n Outcomes):**
*Let $O_1:1, O_2:1, \ldots, O_n:1$ be the odds offered for n mutually exclusive outcomes, and let N be the total amount to bet. The optimal bet amount on outcome i, denoted by $x_i^*$, and the guaranteed profit, denoted by $p^*$, are given by:*

$$x_i^* = \frac{N}{\sum_{j=1}^n \frac{1}{1+O_j}} \cdot \frac{1}{1+O_i}$$

$$p^* = N \left(\frac{\prod_{i=1}^n (1+O_i)}{\sum_{j=1}^n \prod_{k \neq j} (1+O_k)} - 1\right)$$


The `calculate_optimal_bets` function below implements these formulas. Here's what it does:

1. It takes two inputs:
   - `odds`: A list or array of odds for each outcome. Odds must be of the form $O_i:1$.
   - `total_money`: The total amount of money available for betting

2. It calculates the optimal bet for each outcome using the formula for $x_i^*$

3. It calculates the guaranteed profit using the formula for $p^*$

4. It returns a tuple containing:
   - An array of optimal bet amounts for each outcome
   - The guaranteed profit


In [ ]:
import numpy as np

def calculate_optimal_bets(odds, total_money):

    # Convert odds to numpy array for easier calculations
    odds = np.array(odds)

    # Calculate the denominator for optimal bets formula
    denominator = np.sum(1 / (1 + odds))

    # Calculate optimal bets for each outcome
    optimal_bets = total_money / denominator * (1 / (1 + odds))

    # Calculate the guaranteed profit
    profit = total_money * (np.prod(1 + odds) / np.sum(np.prod(1 + odds) / (1 + odds)) - 1)

    return optimal_bets, profit

In [ ]:
#example
#0_A:1 Odds=a/b to 3/2 to 1

odds = [1.0, 2.]
total_money = 100

optimal_bets, profit = calculate_optimal_bets(odds, total_money)

print(f"Odds: {odds}")
print(f"Total money: ${total_money}")
print(f"Optimal bets: {optimal_bets}")
print(f"Guaranteed profit: ${profit:.2f}")



# Probability to odds and odds to probability conversion

In [ ]:
def probability_to_odds(probability):
    """
    Convert probability to odds.

    Args:
    probability (float): A value between 0 and 1 representing the probability.

    Returns:
    float: The corresponding odds.
    """
    if probability <= 0 or probability >= 1:
        raise ValueError("Probability must be between 0 and 1")
    return (1 - probability) / probability

def odds_to_probability(odds):
    """
    Convert odds to probability.

    Args:
    odds (float): The odds value.

    Returns:
    float: The corresponding probability between 0 and 1.
    """
    return 1 / (odds + 1)





In [ ]:
# Example usage
prob = 0.2
print("Probability to Odds:")
odds = probability_to_odds(prob)
print(f"Probability: {prob:.2f}, Odds: {odds:.2f}")

print("\nOdds to Probability:")
odds=4.0
prob = odds_to_probability(odds)
print(f"Odds: {odds:.2f}, Probability: {prob:.2f}")


# Introduction to Options


## Option Terminology

Options are financial derivatives that give buyers the right, but not the obligation, to buy or sell an underlying asset at a specified price, known as the strike price, before a certain expiration date.

- **Long Position (Buying an Option):** The buyer of the option holds a long position. They have the right to exercise the option.
- **Short Position (Selling/Writing an Option):** The seller or writer of the option holds a short position. They have the obligation to fulfill the contract if the buyer exercises the option.


- **Call Option:** Gives the holder the right to buy the underlying asset.
- **Put Option:** Gives the holder the right to sell the underlying asset.

- **In the Money:**
  - For a call option, when the underlying asset price is above the strike price.
  - For a put option, when the underlying asset price is below the strike price.
- **At the Money:** The underlying asset's price is equal to the strike price.
- **Out of the Money:**
  - For a call option, when the underlying asset price is below the strike price.
  - For a put option, when the underlying asset price is above the strike price.

- **Expiration Date:** The date on which the option expires. If the option is not exercised by this date, it becomes worthless.

- **Premium:** The price paid by the buyer to the seller to acquire the rights that the option grants.


The profit for options can be calculated as follows:

- **Call Option Profit:**
  $$\text{Profit} = \max(S - E, 0) - \text{Premium}$$
  Where $S$ is the spot price of the underlying asset and $E$ is the strike price.

- **Put Option Profit:**
  $$\text{Profit} = \max(E - S, 0) - \text{Premium}$$
  Where $S$ is the spot price of the underlying asset and $E$ is the strike price.

## Option Strategies


In the context of options trading, a portfolio can be constructed using various combinations of call and put options. The profit or loss of these portfolios is evaluated at expiration time \( t = T \). Below, we discuss several types of option portfolios, their profit calculations, and the market expectations that might lead an investor to adopt each strategy.

Let $ C_E $ denote a call option with strike price $ E $, and $ P_E $ denote a put option with the same strike price. A negative sign (e.g., $ -C_E $) indicates a short position in the option.



1. **Long Call**: $\text{Port} = C_E $

2. **Short Call**: $ \text{Port} = -C_E $

3. **Long Put**: $ \text{Port} = P_E $
   
4. **Short Put**: $ \text{Port} = -P_E $
  
5. **Long Straddle**: $ \text{Port} = P_E + C_E $
  
6. **Short Straddle**: $ \text{Port} = -P_E - C_E $
   
7. **Long Strangle**: $ \text{Port} = P_{E1} + C_{E2} $
  
8. **Short Butterfly**: $ \text{Port} = -C_{E1} + 2C_{E2} - C_{E3} $
  
9. **Long Butterfly**: $ \text{Port} = C_{E1} - 2C_{E2} + C_{E3} $
   
10. **Short Condor**: $ \text{Port} = -C_{E1} + C_{E2} + C_{E3} - C_{E4} $

11. **Long Condor**: $ \text{Port} = C_{E1} - C_{E2} - C_{E3} + C_{E4} $

### calculate_option_payoff function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_option_payoff(strategy, strike, premium, stock_prices):
    """
    Calculate the payoff for a single option based on the strategy.
    """
    if strategy == "long_call":
        payoff = np.maximum(stock_prices - strike, 0) - premium
    elif strategy == "short_call":
        payoff = -np.maximum(stock_prices - strike, 0) + premium
    elif strategy == "long_put":
        payoff = np.maximum(strike - stock_prices, 0) - premium
    elif strategy == "short_put":
        payoff = -np.maximum(strike - stock_prices, 0) + premium
    else:
        raise ValueError("Invalid strategy")
    return payoff

def plot_options_strategies(strategies, stock_price_range):
    """
    Plot the profit/loss for a combination of option strategies.
    """

    stock_prices = np.linspace(stock_price_range[0], stock_price_range[1], 200)
    total_payoff = np.zeros_like(stock_prices)
    plt.figure(figsize=(12, 8))

    # Plotting each strategy's payoff separately
    for i, strategy in enumerate(strategies):
        payoff = calculate_option_payoff(strategy['type'], strategy['strike'], strategy['premium'], stock_prices)
        total_payoff += payoff
        plt.plot(stock_prices, payoff, label=f"{strategy['type'].replace('_', ' ').title()} (Strike: {strategy['strike']}, Premium: {strategy['premium']})", linestyle='--')

    # Plotting the total portfolio payoff
    plt.plot(stock_prices, total_payoff, label="Total Portfolio Profit", color='black', linewidth=2)

    # Adding annotations for strike prices and break-even points
    for strategy in strategies:
        plt.axvline(x=strategy['strike'], color='grey', linestyle=':', alpha=0.7)
        plt.text(strategy['strike'], 0, f"Strike: {strategy['strike']}", horizontalalignment='center', verticalalignment='bottom')

    # Setting up the graph with labels, legends, and grid
    plt.axhline(0, color='gray', lw=1)
    plt.xlabel("Stock Price at Expiration")
    plt.ylabel("Profit/Loss")
    plt.title("Profit/Loss for Combined Options Strategies")
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:

# Example usage
strategies = [
    {"type": "long_call", "strike":235, "premium": 3.6},

    {"type": "short_call", "strike": 237.5, "premium":1.93},

    {"type": "short_call", "strike":240, "premium": .78},

    {"type": "long_call", "strike":242.5, "premium": .28},

]


plot_options_strategies(strategies, (220, 250))

## Put Call Parity




The put-call parity  principle in options pricing that explains the relationship between the price of call and put options with the same strike price and expiration date.

The put-call parity equation is given by:

$$ S + P_E(S, t) = C_E(S, t) + Ee^{-r(T-t)} $$

where:

- $ S $ is the current price of the stock (spot price).
- $ P_E(S, t) $ is the price of the put option with strike price $E $ and time $ t $ until expiration.
- $C_E(S, t) $ is the price of the call option with the same strike price and time to expiration as the put.
- $E$ is the strike price of the options.
- $ e^{-r(T-t)}$ is the present value factor for a risk-free bond that pays $ E $ at time $T $, with $ r$ being the continuously compounded risk-free interest rate and $ T-t $ being the time to maturity of the options.

The put-call parity shows that the price of a call option implies a certain fair price for the corresponding put option having the same strike price and expiration, and vice versa, given the current stock price and the risk-free rate.

### Arbitrage Put Call Parity

In [ ]:
import numpy as np

def put_call_parity_arb(C, P, E, S, r, T):
    """
    Check for arbitrage via put-call parity:

        C + E*e^{-rT}  ?  P + S

    - C: Call price
    - P: Put price
    - E: Strike price
    - S: Current stock price
    - r: Annualized risk-free rate (decimal), e.g. 0.05 for 5%
    - T: Time to maturity in years (e.g. 30 days = 30/365)

    Returns:
    - (arbitrage_amount, strategy)
    """
    # Present value of strike
    E_discounted = E * np.exp(-r * T)

    # Left-hand side (LHS) and right-hand side (RHS)
    LHS = C + E_discounted
    RHS = P + S

    if LHS > RHS:
        # LHS is overpriced => Sell Call, Buy Put, Borrow strike discount, Buy Stock
        arbitrage_amount = LHS - RHS
        strategy = (
            f"LHS > RHS by ${arbitrage_amount:.2f}: "
            "Sell the Call, Buy the Put, Borrow the Present Value of Strike, Buy the Stock."
        )
    elif LHS < RHS:
        # RHS is overpriced => Buy Call, Sell Put, Short Stock, Invest strike discount
        arbitrage_amount = RHS - LHS
        strategy = (
            f"RHS > LHS by ${arbitrage_amount:.2f}: "
            "Buy the Call, Sell the Put, Short the Stock, Invest the Present Value of Strike."
        )
    else:
        # No arbitrage
        arbitrage_amount = 0
        strategy = "No arbitrage. LHS == RHS."

    return arbitrage_amount, strategy


In [ ]:
# Given values
C = .8  # Price of the call option
P =  2.85 # Price of the put option
E = 240  # Strike price
S = 237.87  # Current stock price
r = 0.05 # Interest rate
T = 2/365  # Time to expiration in years
# Calculate the arbitrage amount and strategy
arbitrage_amount, strategy = put_call_parity_arb(C, P, E, S, r, T)

print(f"Arbitrage opportunity of ${arbitrage_amount:.2f} exists.")
print("Strategy:", strategy)

In [ ]:
import yfinance as yf
from datetime import datetime
import pandas as pd
def find_put_call_arbitrage(ticker_symbol, expiration_date, r=0.005, strike_range=10):
    """
    1) Fetch the current stock price from Yahoo Finance.
    2) Fetch call & put data for a given expiration date.
    3) Filter by strikes within ± strike_range of the current stock price.
    4) Merge calls & puts on strike.
    5) Compute T (in years) until expiration_date.
    6) Check put-call parity for each strike to see if there's any arbitrage.

    Returns a formatted DataFrame.
    """
    # 1) Get current stock price
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(period="1d")
    if len(hist) == 0:
        raise ValueError("Could not fetch current stock data. Check ticker or network.")
    S = hist["Close"].iloc[-1]

    # 2) Fetch options chain
    options_chain = ticker.option_chain(expiration_date)
    calls = options_chain.calls
    puts = options_chain.puts

    # 3) Filter strikes
    #    Use .copy() to avoid SettingWithCopyWarning
    calls_filtered = calls.loc[
        (calls["strike"] >= S - strike_range) &
        (calls["strike"] <= S + strike_range)
    ].copy()

    puts_filtered = puts.loc[
        (puts["strike"] >= S - strike_range) &
        (puts["strike"] <= S + strike_range)
    ].copy()

    # Compute mid-prices
    calls_filtered["call_mid"] = (calls_filtered["bid"] + calls_filtered["ask"]) / 2.0
    puts_filtered["put_mid"] = (puts_filtered["bid"] + puts_filtered["ask"]) / 2.0

    calls_relevant = calls_filtered[["strike", "call_mid"]]
    puts_relevant = puts_filtered[["strike", "put_mid"]]

    # Merge on strike
    merged = calls_relevant.merge(puts_relevant, on="strike", how="inner")

    # 4) Compute time to maturity (T in years)
    exp_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    now = datetime.now()
    days_to_expiry = (exp_date - now).days
    T = days_to_expiry / 365.0

    # 5) Check put-call parity
    results = []
    for _, row in merged.iterrows():
        strike = row["strike"]
        C = row["call_mid"]
        P = row["put_mid"]

        # Skip if we don't have valid (non-NaN) prices
        if np.isnan(C) or np.isnan(P):
            continue

        arbitrage_amount, strategy = put_call_parity_arb(C, P, strike, S, r, T)

        E_discounted = strike * np.exp(-r * T)
        LHS = C + E_discounted
        RHS = P + S

        results.append({
            "Strike": strike,
            "Call Price": C,
            "Put Price": P,
            "Stock Price": S,
            "PV(Strike)": E_discounted,
            "LHS": LHS,
            "RHS": RHS,
            "Arbitrage": arbitrage_amount,
            "Strategy": strategy
        })

    df_results = pd.DataFrame(results)
    # Sort by absolute arbitrage amount (descending)
    df_results.sort_values(
        by="Arbitrage",
        key=lambda x: x.abs(),
        ascending=False,
        inplace=True
    )

    # OPTIONAL: Round numeric columns for a cleaner display
    # (Feel free to adjust decimal places as needed)
    numeric_cols = ["Strike", "Call Price", "Put Price", "Stock Price",
                    "PV(Strike)", "LHS", "RHS", "Arbitrage"]
    for col in numeric_cols:
        df_results[col] = df_results[col].round(3)

    return df_results


In [ ]:
df_arbitrage = find_put_call_arbitrage(
         ticker_symbol="NVDA",
         expiration_date="2025-03-21",
         r=0.05,          # 0.5% annual risk-free rate
         strike_range=5   # Only look at strikes within ±$strike_range of the current stock price
     )
print(df_arbitrage)


# Probability Code

## Expected Value and Variance

In [ ]:
def discrete_expected_value_and_variance(probabilities, values):
    """
    Calculates the expected value and variance of a discrete random variable.

    :param probabilities: list or array of probabilities [p_1, p_2, ..., p_n]
    :param values: list or array of corresponding values [x_1, x_2, ..., x_n]
    :return: (E, Var) where E is expected value and Var is variance
    """
    # Check that the lengths match
    if len(probabilities) != len(values):
        raise ValueError("The length of probabilities must match the length of values.")

    # Check that probabilities sum to 1 (within a small tolerance)
    if abs(sum(probabilities) - 1.0) > 1e-8:
        raise ValueError("The sum of the probabilities must be 1.")

    # Compute E(X)
    expected_value = sum(p * x for p, x in zip(probabilities, values))

    # Compute E(X^2)
    expected_value_sq = sum(p * (x**2) for p, x in zip(probabilities, values))

    # Variance = E(X^2) - [E(X)]^2
    variance = expected_value_sq - (expected_value**2)

    return expected_value, variance

P=[1/3, 1/3, 1/3]
x=[1, -1, 0]
expected_value, variance=discrete_expected_value_and_variance(P, x)
print(f"Expected Value: {expected_value}")
print(f"Variance: {variance}")

In [ ]:
import numpy as np
from scipy.integrate import quad

def continuous_expected_value_and_variance(pdf_func, a, b):
    """
    Calculates the expected value and variance of a continuous random variable
    defined on [a, b] using numerical integration.

    :param pdf_func: A Python function that returns the PDF value at x, e.g., f(x).
    :param a: Lower bound of the domain.
    :param b: Upper bound of the domain.
    :return: (E, Var) tuple where
             E   = the expected value (float)
             Var = the variance (float)
    """

    # 1) Check if PDF integrates to 1 on [a, b]
    total_prob, _ = quad(pdf_func, a, b)
    if abs(total_prob - 1.0) > 1e-7:
        raise ValueError(f"PDF does not integrate to 1 over [{a}, {b}] (got {total_prob}).")

    # 2) Compute E(X) = ∫ x f(x) dx
    def integrand_for_mean(x):
        return x * pdf_func(x)

    E, _ = quad(integrand_for_mean, a, b)

    # 3) Compute E(X^2) = ∫ x^2 f(x) dx
    def integrand_for_mean_sq(x):
        return (x**2) * pdf_func(x)

    E_sq, _ = quad(integrand_for_mean_sq, a, b)

    # 4) Compute Variance = E(X^2) - [E(X)]^2
    variance = E_sq - (E**2)

    return E, variance

def pdf(x):
  return 2*x

E, var=continuous_expected_value_and_variance(pdf, 0, 1)
print(f"Expected Value: {E}")
print(f"Variance: {var}")


## Normal Distribtuion

In [ ]:
import math
import numpy as np
from scipy.stats import norm

def normal_prob(b, mu, sigma_sq):
    """
    Calculate P(X <= b) where X ~ N(mu, sigma_sq).

    :param b: Upper limit (scalar).
    :param mu: Mean of the normal distribution (scalar).
    :param sigma_sq: Variance of the normal distribution (scalar).
    :return: Probability P(X <= b).
    """
    if sigma_sq <= 0:
        raise ValueError("Variance sigma_sq must be positive.")

    # Standard deviation
    sigma = math.sqrt(sigma_sq)

    # Standardize and use the standard normal CDF
    z = (b - mu) / sigma
    p = norm.cdf(z)
    return p

# Suppose X ~ N(mu=2, sigma^2=4), i.e., sigma=2
# We want P(X <= b) for b = 3
b_val = 3
mu_val = 2
sigma_sq_val = 4

probability = normal_prob(b_val, mu_val, sigma_sq_val)
print("Normal Distribution Example")
print(f"X ~ N(mu={mu_val}, sigma^2={sigma_sq_val})")
print(f"Probability P(X <= {b_val}) = {probability}")

## CLT


In [ ]:
def clt(mu, sigma_sq, n):
    """
    Demonstrate the Central Limit Theorem for i.i.d. random variables X_i ~ (mu, sigma_sq).

    - S_n = X_1 + ... + X_n
      E[S_n] = n * mu
      Var(S_n) = n * sigma_sq

    - S_n / n
      E[S_n / n] = mu
      Var(S_n / n) = sigma_sq / n


    :param mu:       Mean of each X_i.
    :param sigma_sq: Variance of each X_i.
    :param n:        Number of i.i.d. trials.
    """

    if sigma_sq <= 0:
        raise ValueError("Variance must be positive.")
    if n <= 0:
        raise ValueError("Number of trials n must be positive.")

    # Mean and variance of S_n
    mean_Sn = n * mu
    var_Sn = n * sigma_sq

    # Mean and variance of S_n / n
    mean_SnOverN = mu
    var_SnOverN  = sigma_sq / n

    return mean_Sn, var_Sn, mean_SnOverN, var_SnOverN

In [ ]:
#Example:

mu= 2.0       # mean of each X_i
var = 4.0 # variance of each X_i
n = 30         # number of i.i.d. trials


mean_Sn, var_Sn, mean_SnOverN, var_SnOverN  = clt(mu, var, n)
print(f"Sample mean:{mean_SnOverN}" )
print(f"Sample Variance:{var_SnOverN}")
print(f"Sum Mean:{mean_Sn}")
print(f"Sum Variance:{var_Sn}")

In [ ]:
'''
list of functions for far (Midterm 1):

calculate_optimal_bets(odds, total_money)
probability_to_odds(probability)
odds_to_probability(odds)
calculate_option_payoff(strategy, strike, premium, stock_prices)
plot_options_strategies(strategies, stock_price_range)
put_call_parity_arb(C, P, E, S, r, T)
discrete_expected_value_and_variance(probabilities, values)
continuous_expected_value_and_variance(pdf_func, a, b)
normal_prob(b, mu, sigma_sq)
clt(mu, sigma_sq, n)

'''



# Probelm Solving


## Problem 1
A stock is currently priced at \$100 per share. By tomorrow, its price will either drop to \$50 or rise to \$200.  

Meanwhile, a casino offers bets on the price change at 1:1 odds.  

What’s the maximum arbitrage? You have $100 and you can buy fractional shares.

In [ ]:
# put x in Stock and 100-x in casino betting on decrease
x=200/3.5
P1= x-(100- x)
P2=((100-x)-.5*x)
print(x, P1, P2)

## Problem 2


Suppose the PDF is given by:

$$
f(x) = Cx^2, \quad \text{for } 1 \leq x \leq 2
$$

and zero otherwise.

 (a)
Find the value of $C $.

(b)
Find the likelihood that a point selected at random with respect to this PDF is between $1.5 $ and $ 2 $.

(c)
With $X(x) = x $, find $ E(X) $ and $ \text{Var}(X) $.


In [ ]:
c=3/7
def pdf_func(x):
  return c*x**2
E, Var=continuous_expected_value_and_variance(pdf_func, a=1, b=2)

print(f"Expected Value: {E}")
print(f"Variance: {Var}")

## Problem 3


Suppose the only game in town involves flipping a fair coin (so Heads and Tails  
are equally likely) with a bet of $ x $.  

- If Heads comes up, the payoff is $ 0.9x $.  
- If Tails comes up, you lose the full amount $ x $.  

You have $\$10,000$ and must win at least $\$5,000$ by tomorrow morning to pay off a debt to a mean dude.

(a)
Compute the likelihood of winning at least $\$5,000$ by making a single bet of \$10,000.

(b)
Compute the likelihood of winning at least $\$1,000$ by playing the game $10,000$ times and betting \$1 each time.  

What is the likelihood of not losing money?



In [ ]:
# compute E, Var of X
n=10000
b=1000
probabilities=[1/2, 1/2]
values=[.9, -1.]
E, var=discrete_expected_value_and_variance(probabilities, values)

mu, variance, _, _=clt(E, Var, n)

p=normal_prob(b, mu, variance)

print(1-p)


## Problem 4



A math of finance student needs to take out a one-year loan of \$14,700. The bank offers a loan rate of 7% for students.

They also notice the following information about Apple (AAPL) stock and options:

- **Current AAPL stock price**: \$150  
- **1-year European call option on AAPL with strike price \$150**: \$15  
- **1-year European put option on AAPL with strike price \$150**: \$12  

What is the best interest rate the student can get?


## Problem 5

You can buy and sell oranges in a farmer's market under the following conditions:

- **Market Trading:**
  - The current price of an orange at time **T₀** is **$10**.
  - The price at time **T₁** is **unknown**.

- **European Call Option:**
  - Cost: **\$0.50** per option.
  - Strike price: **\$9**.
  - Expiration: **T₁**.

- **Neighbor Lending/Borrowing:**
  - Your friendly neighbor can **lend you one orange** or **borrow one from you** at **T₀**.
  - If you borrow an orange, you must **return one orange at T₁**.

- **Bank Loans:**
  - You can **lend or borrow money** with a **10\% interest rate**.

Can you find arbitrage?
